In [1]:
import pandas as pd
import numpy as np
import joblib
import argparse
from sklearn.ensemble import RandomForestClassifier  # random forest model
from sklearn.model_selection import GridSearchCV
from sklearn import metrics  # to calculate the accuracy of the model
from sklearn.metrics import (precision_recall_curve, PrecisionRecallDisplay, auc)
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import permutation_importance
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
from imblearn.under_sampling import RandomUnderSampler
import random
from sklearn.ensemble import RandomForestClassifier  # random forest model
from sklearn.model_selection import GridSearchCV
from sklearn import metrics  # to calculate the accuracy of the model and confusion matrix
from sklearn.metrics import (precision_recall_curve, PrecisionRecallDisplay, auc)
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import permutation_importance
from sklearn import tree
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
from imblearn.under_sampling import RandomUnderSampler
import random

# APPLICATION of RANDOM FOREST best models 

The feature **table** has to be built with the **same column names** and **column order** as the **trained model**

In [2]:
models_name = ["clumps", "std", "clumps_std"]
base_dir = "/home/giulia/Workspace/PhytoPhD/period_abroad/"

### on 7 CaPmali strains

In [10]:
capms = [5, 6, 7, 11, 13, 40, 44]

## unique feature table 
for mod in models_name:
    loaded_model = joblib.load(f"{base_dir}random_forest/on_dataset20230224/revision_prosite2230314/{mod}_features/rf_best_model.pkl")
    for s in capms:

        table_base_dir = f"{base_dir}random_forest/on_7_strains/from_drive20230303/"
        origin_data_pred_df = pd.read_csv(f"{table_base_dir}capm{s}/pred/feature_table_capm{s}_{mod}.tsv",
                                          sep="\t",
                                          header=0)

        original_protein_names = origin_data_pred_df["name"] 
        original_protein_seq_id = origin_data_pred_df["seq_id"]

        data_pred_df = origin_data_pred_df[origin_data_pred_df.columns[1:]] # df without seq_id == ready for predictions
        
        # apply the model
        clf = loaded_model.predict(data_pred_df[list(data_pred_df.columns)[1:]])
        list_clf = list(clf)
        df_clf = pd.DataFrame(clf, columns=["clf"])
        
        # save the predictions
        df_clf.to_csv(f"{table_base_dir}capm{s}/norm_prosite_clumps20230314/model_{mod}_on_dataset20230224/rf_best_model_predictions.tsv", 
                      sep="\t", 
                      index=False)

        # probability of class assignment 
        prob = loaded_model.predict_proba(data_pred_df[list(data_pred_df.columns)[1:]])
        df_prob = pd.DataFrame(prob, columns=["effector_prob", "non_effector_prob"])
        df_prob["seq_id"] = list(original_protein_seq_id)
        df_prob.to_csv(f"{table_base_dir}capm{s}/norm_prosite_clumps20230314/model_{mod}_on_dataset20230224/probability_of_predictions.tsv",
                       sep="\t",
                       index=False)
        
        # save all effectors sequences
        eff = df_prob[df_prob["effector_prob"] >= 0.5]
        with open(f"{table_base_dir}capm{s}/norm_prosite_clumps20230314/model_{mod}_on_dataset20230224/effector_predictions_{mod}.fasta", 
              "w") as filtered_seq:
            for record in SeqIO.parse(f"{base_dir}datasets/7_strains/CaPm{s}_prodigal_faa.fasta", "fasta"):
                if record.id in list(eff["seq_id"]):
                    filtered_seq.write(f">{record.description}\n{str(record.seq)}\n")

        ## save effectors with probaility = confidence of prediction > 90 into a fasta file
        highly_eff = df_prob[df_prob["effector_prob"] > 0.9]
        with open(f"{table_base_dir}capm{s}/norm_prosite_clumps20230314/model_{mod}_on_dataset20230224/effector_predictions_filt90.fasta", 
                  "w") as filtered_seq:
            for record in SeqIO.parse(f"{base_dir}datasets/7_strains/CaPm{s}_prodigal_faa.fasta", "fasta"):
                if record.id in list(highly_eff["seq_id"]):
                    filtered_seq.write(f">{record.description}\n{str(record.seq)}\n")

        # save the summary of performances 
        with open(f"{table_base_dir}capm{s}/norm_prosite_clumps20230314/model_{mod}_on_dataset20230224/prediction_summary.txt", 
                  "w") as out:
            count_eff = list_clf.count('effector')
            count_noneff = list_clf.count('non_effector')
            out.write(f"CaPm{s}\nModel: {mod}\nPredicted:\n{count_eff} effectors\n{count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/count_noneff)*100}")

 
        # save the predictions as they are
        df_predictions = pd.DataFrame({"clf": list_clf, "protein_id": origin_data_pred_df["seq_id"]})
        df_predictions.to_csv(f"{table_base_dir}capm{s}/norm_prosite_clumps20230314/model_{mod}_on_dataset20230224/predictions.tsv", 
                              sep="\t",
                              index=False)

        print(f"""CaPm{s}\nModel: {mod}\nPredicted:\n{count_eff} effectors
        {count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/len(origin_data_pred_df))*100}%""")



CaPm5
Model: clumps
Predicted:
144 effectors
        375 non-effectors
Proportion of effectors/total proteins: 27.74566473988439%
CaPm6
Model: clumps
Predicted:
182 effectors
        436 non-effectors
Proportion of effectors/total proteins: 29.449838187702266%
CaPm7
Model: clumps
Predicted:
127 effectors
        364 non-effectors
Proportion of effectors/total proteins: 25.865580448065174%
CaPm11
Model: clumps
Predicted:
168 effectors
        388 non-effectors
Proportion of effectors/total proteins: 30.215827338129497%
CaPm13
Model: clumps
Predicted:
106 effectors
        333 non-effectors
Proportion of effectors/total proteins: 24.145785876993166%
CaPm40
Model: clumps
Predicted:
161 effectors
        385 non-effectors
Proportion of effectors/total proteins: 29.48717948717949%
CaPm44
Model: clumps
Predicted:
181 effectors
        400 non-effectors
Proportion of effectors/total proteins: 31.153184165232357%
CaPm5
Model: std
Predicted:
253 effectors
        266 non-effectors
Proportion of

In [11]:
# SUMMARY 

# per ogni cartella contenente le predizioni dei pm, costruire un dizionario che sara' salvato in un df 

summary_dict = {"strain": [],
                "model": [],
                "total_proteins": [],
                "pred_eff": [], 
                "pred_noneff": [],
                "eff / tot_proteins": [],
                "eff_prob_equal100": [],
                "eff_prob_95": [],
                "eff_90_prob_95": [],
                "eff_80_prob_90": [],
                "eff_70_prob_80": [],
                "eff_60_prob_70": [],
                "eff_50_prob_60": [],
                "eff_prob_equal50": [],
                "eff_50_prob": []}

index_df = []
for mod in models_name:
    for s in capms: 
        df_prob = pd.read_csv(f"{table_base_dir}capm{s}/norm_prosite_clumps20230314/model_{mod}_on_dataset20230224/probability_of_predictions.tsv", 
                              sep="\t", 
                              header=0)
        list_clf = list(pd.read_csv(f"{table_base_dir}capm{s}/norm_prosite_clumps20230314/model_{mod}_on_dataset20230224/rf_best_model_predictions.tsv", 
                               sep="\t",
                              header=0)["clf"])

        total_proteins = len(df_prob)
        count_eff = list_clf.count("effector")
        count_noneff = list_clf.count("non_effector")
        eff_100 = df_prob[df_prob["effector_prob"] == 1]
        eff_95 = df_prob[df_prob["effector_prob"] >= 0.95] # >=95%
        eff_90_95 = df_prob[(df_prob["effector_prob"] < 0.95) & (df_prob["effector_prob"] >= 0.9)]
        eff_80_90 = df_prob[(df_prob["effector_prob"] < 0.9) & (df_prob["effector_prob"] >= 0.8)] 
        eff_70_80 = df_prob[(df_prob["effector_prob"] < 0.8) & (df_prob["effector_prob"] >= 0.7)]
        eff_60_70 = df_prob[(df_prob["effector_prob"] < 0.7) & (df_prob["effector_prob"] >=0.6)]
        eff_50_60 = df_prob[(df_prob["effector_prob"] < 0.6) & (df_prob["effector_prob"] >= 0.5)]
        eff_50 = df_prob[df_prob["effector_prob"] == 0.5]
        eff_low_50 = df_prob[df_prob["effector_prob"] < 0.5]
        summary_dict["strain"].append(f"CaPm{s}")
        summary_dict["model"].append(mod)
        summary_dict["total_proteins"].append(total_proteins)
        summary_dict["pred_eff"].append(count_eff)
        summary_dict["pred_noneff"].append(count_noneff)
        summary_dict["eff / tot_proteins"].append(f"{count_eff/len(origin_data_pred_df)*100:.1f}%")
        summary_dict["eff_prob_equal100"].append(f"{len(eff_100)} ({(len(eff_100)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_prob_95"].append(f"{len(eff_95)} ({(len(eff_95)/len(origin_data_pred_df))*100:.1f}%)") # >95
        summary_dict["eff_90_prob_95"].append(f"{len(eff_90_95)} ({(len(eff_90_95)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_80_prob_90"].append(f"{len(eff_80_90)} ({(len(eff_80_90)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_70_prob_80"].append(f"{len(eff_70_80)} ({(len(eff_70_80)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_60_prob_70"].append(f"{len(eff_60_70)} ({(len(eff_60_70)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_50_prob_60"].append(f"{len(eff_50_60)} ({(len(eff_50_60)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_prob_equal50"].append(f"{len(eff_50)} ({(len(eff_50)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_50_prob"].append(f"{len(eff_low_50)} ({(len(eff_low_50)/len(origin_data_pred_df))*100:.1f}%)")
    
summary_df = pd.DataFrame(summary_dict)

output_summary = f"{base_dir}random_forest/on_7_strains/from_drive20230303/norm_prosite_clumps20230314_summary_rf_predictions.tsv"
with open(output_summary, "w") as output:
    output.write(f"""#Random_Forest\n#Stratified5foldCV\n#Shuffle=Truex1\n#GridSearch_of_best_n_estimators=best_n_trees\n""")
    summary_df.to_csv(output, sep="\t", index=False)

## On single feature table

In [146]:
## unique feature table 

for mod in models_name:
    loaded_model = joblib.load(f"{base_dir}random_forest/on_dataset20230224/{mod}_features/rf_best_model.pkl")

    table_base_dir = f"{base_dir}random_forest/"
    origin_data_pred_df = pd.read_csv(f"{table_base_dir}on_AT/pred/feature_table_{mod}.tsv",
                                      sep="\t",
                                      header=0)

    original_protein_names = origin_data_pred_df["name"] 
    original_protein_seq_id = origin_data_pred_df["seq_id"]

    data_pred_df = origin_data_pred_df[origin_data_pred_df.columns[1:]] # df without seq_id == ready for predictions

    # apply the model
    clf = loaded_model.predict(data_pred_df[list(data_pred_df.columns)[1:]])
    list_clf = list(clf)
    df_clf = pd.DataFrame(clf, columns=["clf"])

    # save the predictions
    df_clf.to_csv(f"{base_dir}/random_forest/on_AT/rf_best_model_{mod}_predictions.tsv", 
                  sep="\t", 
                  index=False)

    # probability of class assignment 
    prob = loaded_model.predict_proba(data_pred_df[list(data_pred_df.columns)[1:]])
    df_prob = pd.DataFrame(prob, columns=["effector_prob", "non_effector_prob"])
    df_prob["seq_id"] = list(original_protein_seq_id)
    df_prob.to_csv(f"{base_dir}/random_forest/on_AT/probability_of_predictions_{mod}.tsv",
                   sep="\t",
                   index=False)
    


    ## save effectors with probaility = confidence of prediction > 90 into a fasta file
    highly_eff = df_prob[df_prob["effector_prob"] > 0.9]
    with open(f"{base_dir}/random_forest/on_AT/effector_predictions_{mod}_filt90.fasta", 
              "w") as filtered_seq:
        for record in SeqIO.parse(f"{base_dir}datasets/uniproteomes/capmali_os482235_uniproteome_complete.fasta", "fasta"):
            if record.id in list(highly_eff["seq_id"]):
                filtered_seq.write(f">{record.description}\n{str(record.seq)}\n")
                
    

    # save the summary of performances 
    with open(f"{base_dir}/random_forest/on_AT/prediction_{mod}_summary.txt", 
              "w") as out:
        count_eff = list_clf.count('effector')
        count_noneff = list_clf.count('non_effector')
        out.write(f"AT\nModel: {mod}\nPredicted:\n{count_eff} effectors\n{count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/count_noneff)*100}")


    # save the predictions as they are
    df_predictions = pd.DataFrame({"clf": list_clf, "protein_id": origin_data_pred_df["seq_id"]})
    df_predictions.to_csv(f"{base_dir}/random_forest/on_AT/predictions_{mod}.tsv", 
                          sep="\t",
                          index=False)

    print(f"""AT\nModel: {mod}\nPredicted:\n{count_eff} effectors
    {count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/len(origin_data_pred_df))*100}%""")



AT
Model: clumps
Predicted:
96 effectors
    352 non-effectors
Proportion of effectors/total proteins: 21.428571428571427%
AT
Model: std
Predicted:
96 effectors
    352 non-effectors
Proportion of effectors/total proteins: 21.428571428571427%
AT
Model: clumps_std
Predicted:
94 effectors
    354 non-effectors
Proportion of effectors/total proteins: 20.982142857142858%


# ON Flavescence Doree

In [152]:
fds = ["Chard_FD", "Pgris_SVinc_FD"]
## unique feature table 

for mod in models_name:
    for f in fds:
        loaded_model = joblib.load(f"{base_dir}random_forest/on_dataset20230224/{mod}_features/rf_best_model.pkl")

        table_base_dir = f"{base_dir}random_forest/"
        origin_data_pred_df = pd.read_csv(f"{table_base_dir}on_FDs_20230307/{f}/pred/feature_table_{mod}.tsv",
                                          sep="\t",
                                          header=0)

        original_protein_names = origin_data_pred_df["name"] 
        original_protein_seq_id = origin_data_pred_df["seq_id"]

        data_pred_df = origin_data_pred_df[origin_data_pred_df.columns[1:]] # df without seq_id == ready for predictions

        # apply the model
        clf = loaded_model.predict(data_pred_df[list(data_pred_df.columns)[1:]])
        list_clf = list(clf)
        df_clf = pd.DataFrame(clf, columns=["clf"])

        # save the predictions
        df_clf.to_csv(f"{base_dir}/random_forest/on_FDs_20230307/{f}/rf_best_model_{mod}_predictions.tsv", 
                      sep="\t", 
                      index=False)

        # probability of class assignment 
        prob = loaded_model.predict_proba(data_pred_df[list(data_pred_df.columns)[1:]])
        df_prob = pd.DataFrame(prob, columns=["effector_prob", "non_effector_prob"])
        df_prob["seq_id"] = list(original_protein_seq_id)
        df_prob.to_csv(f"{base_dir}/random_forest/on_FDs_20230307/{f}/probability_of_predictions_{mod}.tsv",
                       sep="\t",
                       index=False)

        ## save effectors with probaility = confidence of prediction > 90 into a fasta file
        highly_eff = df_prob[df_prob["effector_prob"] > 0.9]
        with open(f"{base_dir}/random_forest/on_FDs_20230307/{f}/effector_predictions_{mod}_filt90.fasta", 
                  "w") as filtered_seq:
            for record in SeqIO.parse(f"{base_dir}datasets/FDs/{f}_cds_faa.fasta", "fasta"):
                if record.id in list(highly_eff["seq_id"]):
                    filtered_seq.write(f">{record.description}\n{str(record.seq)}\n")

        # save the summary of performances 
        with open(f"{base_dir}/random_forest/on_FDs_20230307/{f}/prediction_{mod}_summary.txt", 
                  "w") as out:
            count_eff = list_clf.count('effector')
            count_noneff = list_clf.count('non_effector')
            out.write(f"{f}\nModel: {mod}\nPredicted:\n{count_eff} effectors\n{count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/count_noneff)*100}")


        # save the predictions as they are
        df_predictions = pd.DataFrame({"clf": list_clf, "protein_id": origin_data_pred_df["seq_id"]})
        df_predictions.to_csv(f"{base_dir}/random_forest/on_FDs_20230307/{f}/predictions_{mod}.tsv", 
                              sep="\t",
                              index=False)

        print(f"""{f}\nModel: {mod}\nPredicted:\n{count_eff} effectors
        {count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/len(origin_data_pred_df))*100}%""")


Chard_FD
Model: clumps
Predicted:
111 effectors
        494 non-effectors
Proportion of effectors/total proteins: 18.34710743801653%
Pgris_SVinc_FD
Model: clumps
Predicted:
116 effectors
        612 non-effectors
Proportion of effectors/total proteins: 15.934065934065933%
Chard_FD
Model: std
Predicted:
110 effectors
        495 non-effectors
Proportion of effectors/total proteins: 18.181818181818183%
Pgris_SVinc_FD
Model: std
Predicted:
116 effectors
        612 non-effectors
Proportion of effectors/total proteins: 15.934065934065933%
Chard_FD
Model: clumps_std
Predicted:
118 effectors
        487 non-effectors
Proportion of effectors/total proteins: 19.50413223140496%
Pgris_SVinc_FD
Model: clumps_std
Predicted:
119 effectors
        609 non-effectors
Proportion of effectors/total proteins: 16.346153846153847%


In [153]:
# SUMMARY 

# per ogni cartella contenente le predizioni dei FD, costruire un dizionario che sara' salvato in un df 
fds = ["Chard_FD", "Pgris_SVinc_FD"]
summary_dict = {"strain": [],
                "model": [],
                "total_proteins": [],
                "pred_eff": [], 
                "pred_noneff": [],
                "eff / tot_proteins": [],
                "eff_prob_equal100": [],
                "eff_prob_95": [],
                "eff_90_prob_95": [],
                "eff_80_prob_90": [],
                "eff_70_prob_80": [],
                "eff_60_prob_70": [],
                "eff_50_prob_60": [],
                "eff_prob_equal50": [],
                "eff_50_prob": []}

index_df = []
for mod in models_name:
    for f in fds: 
        df_prob = pd.read_csv(f"{table_base_dir}on_FDs_20230307/{f}/probability_of_predictions_{mod}.tsv", 
                              sep="\t", 
                              header=0)
        list_clf = list(pd.read_csv(f"{table_base_dir}on_FDs_20230307/{f}/rf_best_model_{mod}_predictions.tsv", 
                               sep="\t",
                              header=0)["clf"])

        total_proteins = len(df_prob)
        count_eff = list_clf.count("effector")
        count_noneff = list_clf.count("non_effector")
        eff_100 = df_prob[df_prob["effector_prob"] == 1]
        eff_95 = df_prob[df_prob["effector_prob"] >= 0.95] # >=95%
        eff_90_95 = df_prob[(df_prob["effector_prob"] < 0.95) & (df_prob["effector_prob"] >= 0.9)]
        eff_80_90 = df_prob[(df_prob["effector_prob"] < 0.9) & (df_prob["effector_prob"] >= 0.8)] 
        eff_70_80 = df_prob[(df_prob["effector_prob"] < 0.8) & (df_prob["effector_prob"] >= 0.7)]
        eff_60_70 = df_prob[(df_prob["effector_prob"] < 0.7) & (df_prob["effector_prob"] >=0.6)]
        eff_50_60 = df_prob[(df_prob["effector_prob"] < 0.6) & (df_prob["effector_prob"] >= 0.5)]
        eff_50 = df_prob[df_prob["effector_prob"] == 0.5]
        eff_low_50 = df_prob[df_prob["effector_prob"] < 0.5]
        summary_dict["strain"].append(f"{f}")
        summary_dict["model"].append(mod)
        summary_dict["total_proteins"].append(total_proteins)
        summary_dict["pred_eff"].append(count_eff)
        summary_dict["pred_noneff"].append(count_noneff)
        summary_dict["eff / tot_proteins"].append(f"{count_eff/len(origin_data_pred_df)*100:.1f}%")
        summary_dict["eff_prob_equal100"].append(f"{len(eff_100)} ({(len(eff_100)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_prob_95"].append(f"{len(eff_95)} ({(len(eff_95)/len(origin_data_pred_df))*100:.1f}%)") # >95
        summary_dict["eff_90_prob_95"].append(f"{len(eff_90_95)} ({(len(eff_90_95)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_80_prob_90"].append(f"{len(eff_80_90)} ({(len(eff_80_90)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_70_prob_80"].append(f"{len(eff_70_80)} ({(len(eff_70_80)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_60_prob_70"].append(f"{len(eff_60_70)} ({(len(eff_60_70)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_50_prob_60"].append(f"{len(eff_50_60)} ({(len(eff_50_60)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_prob_equal50"].append(f"{len(eff_50)} ({(len(eff_50)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_50_prob"].append(f"{len(eff_low_50)} ({(len(eff_low_50)/len(origin_data_pred_df))*100:.1f}%)")
    
summary_df = pd.DataFrame(summary_dict)

output_summary = f"{table_base_dir}on_FDs_20230307/summary_rf_predictions_FD.tsv"
with open(output_summary, "w") as output:
    output.write(f"""#Random_Forest\n#Stratified5foldCV\n#Shuffle=Truex1\n#GridSearch_of_best_n_estimators=best_n_trees\n""")
    summary_df.to_csv(output, sep="\t", index=False)

# APPLICATION TO PUBLIC PHYTOPROTEOMES

In [24]:
# remove the effector proteins that are already analyzed by the RF model, for each proteome
proteomes = ["CaPoryzae_strainMbita1", "CaPoryzae_strainNGS-S10", "CaPphoenicium_strainChiP", 
             "CaPphoenicium_strainSA213", "CaPpruni_strainCX", "Maize_bushy_stant"]


for mod in models_name:
    for p in proteomes:
        tmp_remove = []
        table_base_dir = f"{base_dir}random_forest/on_{p}/{mod}_features/"
        origin_data_pred_df = pd.read_csv(f"{table_base_dir}feature_table_{mod}.tsv",
                                          sep="\t",
                                          header=0)
        p_sequences = SeqIO.parse(f"{base_dir}datasets/dataset_20230224/uniprot_eff_ref20230223.fasta", "fasta")
        for record in p_sequences:
            if str(record.id) in list(origin_data_pred_df["seq_id"]):
                tmp_remove.append(origin_data_pred_df["seq_id"][origin_data_pred_df["seq_id"] == str(record.id)].index[0])
            else:
                pass
        applicable_data_pred_df = origin_data_pred_df.drop(tmp_remove, axis=0)
        applicable_data_pred_df.to_csv(f"{table_base_dir}feature_table_{mod}.tsv",
                                          sep="\t",
                                          index=False)

In [7]:
proteomes = ["CaPoryzae_strainMbita1", "CaPoryzae_strainNGS-S10", "CaPphoenicium_strainChiP", 
             "CaPphoenicium_strainSA213", "CaPpruni_strainCX", "Maize_bushy_stant"]
pred_eff = []
## unique feature table 
for mod in models_name:
    loaded_model = joblib.load(f"{base_dir}random_forest/on_dataset20230224/{mod}_features/rf_best_model.pkl")
    for p in proteomes:

        table_base_dir = f"{base_dir}random_forest/on_CaPproteomes/on_{p}/{mod}_features/"
        origin_data_pred_df = pd.read_csv(f"{table_base_dir}feature_table_{mod}.tsv",
                                          sep="\t",
                                          header=0)        

        original_protein_names = origin_data_pred_df["name"] 
        original_protein_seq_id = origin_data_pred_df["seq_id"]

        data_pred_df = origin_data_pred_df[origin_data_pred_df.columns[1:]] # df without seq_id == ready for predictions
        
        # apply the model
        clf = loaded_model.predict(data_pred_df[list(data_pred_df.columns)[1:]])
        list_clf = list(clf)
        df_clf = pd.DataFrame(clf, columns=["clf"])
        
        # save the predictions
        df_clf.to_csv(f"{table_base_dir}rf_best_model_predictions.tsv", 
                      sep="\t", 
                      index=False)

        # probability of class assignment 
        prob = loaded_model.predict_proba(data_pred_df[list(data_pred_df.columns)[1:]])
        df_prob = pd.DataFrame(prob, columns=["effector_prob", "non_effector_prob"])
        df_prob["seq_id"] = list(original_protein_seq_id)
        df_prob.to_csv(f"{table_base_dir}probability_of_predictions.tsv",
                       sep="\t",
                       index=False)
        
        # save all effectors sequences
        eff = df_prob[df_prob["effector_prob"] >= 0.5]
        with open(f"{table_base_dir}effector_predictions_{mod}.fasta", 
              "w") as filtered_seq:
            for record in SeqIO.parse(f"{base_dir}datasets/CaPproteomes/{p}/{p}.fasta", "fasta"):
                if record.id in list(eff["seq_id"]):
                    filtered_seq.write(f">{record.description}\n{str(record.seq)}\n")

#         ## save effectors with probaility = confidence of prediction > 90 into a fasta file
#         highly_eff = df_prob[df_prob["effector_prob"] > 0.9]
#         with open(f"{table_base_dir}capm{p}/norm_prosite_clumps20230314/model_{mod}_on_dataset20230224/effector_predictions_filt90.fasta", 
#                   "w") as filtered_seq:
#             for record in SeqIO.parse(f"{base_dir}datasets/7_strains/CaPm{p}_prodigal_faa.fasta", "fasta"):
#                 if record.id in list(highly_eff["seq_id"]):
#                     filtered_seq.write(f">{record.description}\n{str(record.seq)}\n")

        # save the summary of performances 
        with open(f"{table_base_dir}prediction_summary.txt", 
                  "w") as out:
            count_eff = list_clf.count('effector')
            count_noneff = list_clf.count('non_effector')
            out.write(f"{p}\nModel: {mod}\nPredicted:\n{count_eff} effectors\n{count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/count_noneff)*100}")

 
        # save the predictions as they are
        df_predictions = pd.DataFrame({"clf": list_clf, "protein_id": origin_data_pred_df["seq_id"]})
        df_predictions.to_csv(f"{table_base_dir}predictions.tsv", 
                              sep="\t",
                              index=False)

        print(f"""{p}\nModel: {mod}\nPredicted:\n{count_eff} effectors
        {count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/len(origin_data_pred_df))*100}%""")
        pred_eff.append((count_eff/len(origin_data_pred_df))*100)
print(np.mean(pred_eff))
print(np.std(pred_eff))

CaPoryzae_strainMbita1
Model: clumps
Predicted:
85 effectors
        323 non-effectors
Proportion of effectors/total proteins: 20.833333333333336%
CaPoryzae_strainNGS-S10
Model: clumps
Predicted:
74 effectors
        318 non-effectors
Proportion of effectors/total proteins: 18.877551020408163%
CaPphoenicium_strainChiP
Model: clumps
Predicted:
93 effectors
        426 non-effectors
Proportion of effectors/total proteins: 17.91907514450867%
CaPphoenicium_strainSA213
Model: clumps
Predicted:
43 effectors
        283 non-effectors
Proportion of effectors/total proteins: 13.190184049079754%
CaPpruni_strainCX
Model: clumps
Predicted:
104 effectors
        444 non-effectors
Proportion of effectors/total proteins: 18.97810218978102%
Maize_bushy_stant
Model: clumps
Predicted:
83 effectors
        402 non-effectors
Proportion of effectors/total proteins: 17.11340206185567%
CaPoryzae_strainMbita1
Model: std
Predicted:
83 effectors
        325 non-effectors
Proportion of effectors/total proteins: 

In [27]:
# SUMMARY 

# per ogni cartella contenente le predizioni dei FD, costruire un dizionario che sara' salvato in un df 
proteomes = ["CaPoryzae_strainMbita1", "CaPoryzae_strainNGS-S10", "CaPphoenicium_strainChiP", 
             "CaPphoenicium_strainSA213", "CaPpruni_strainCX", "Maize_bushy_stant"]
summary_dict = {"proteome": [],
                "model": [],
                "total_proteins": [],
                "pred_eff": [], 
                "pred_noneff": [],
                "eff / tot_proteins": [],
                "eff_prob_equal100": [],
                "eff_prob_95": [],
                "eff_90_prob_95": [],
                "eff_80_prob_90": [],
                "eff_70_prob_80": [],
                "eff_60_prob_70": [],
                "eff_50_prob_60": [],
                "eff_prob_equal50": [],
                "eff_50_prob": []}

index_df = []
for mod in models_name:
    for p in proteomes: 
        table_base_dir = f"{base_dir}random_forest/on_{p}/{mod}_features/"
        df_prob = pd.read_csv(f"{table_base_dir}probability_of_predictions.tsv", 
                              sep="\t", 
                              header=0)
        list_clf = list(pd.read_csv(f"{table_base_dir}rf_best_model_predictions.tsv", 
                               sep="\t",
                              header=0)["clf"])

        total_proteins = len(df_prob)
        count_eff = list_clf.count("effector")
        count_noneff = list_clf.count("non_effector")
        eff_100 = df_prob[df_prob["effector_prob"] == 1]
        eff_95 = df_prob[df_prob["effector_prob"] >= 0.95] # >=95%
        eff_90_95 = df_prob[(df_prob["effector_prob"] < 0.95) & (df_prob["effector_prob"] >= 0.9)]
        eff_80_90 = df_prob[(df_prob["effector_prob"] < 0.9) & (df_prob["effector_prob"] >= 0.8)] 
        eff_70_80 = df_prob[(df_prob["effector_prob"] < 0.8) & (df_prob["effector_prob"] >= 0.7)]
        eff_60_70 = df_prob[(df_prob["effector_prob"] < 0.7) & (df_prob["effector_prob"] >=0.6)]
        eff_50_60 = df_prob[(df_prob["effector_prob"] < 0.6) & (df_prob["effector_prob"] >= 0.5)]
        eff_50 = df_prob[df_prob["effector_prob"] == 0.5]
        eff_low_50 = df_prob[df_prob["effector_prob"] < 0.5]
        summary_dict["proteome"].append(f"{p}")
        summary_dict["model"].append(mod)
        summary_dict["total_proteins"].append(total_proteins)
        summary_dict["pred_eff"].append(count_eff)
        summary_dict["pred_noneff"].append(count_noneff)
        summary_dict["eff / tot_proteins"].append(f"{count_eff/len(origin_data_pred_df)*100:.1f}%")
        summary_dict["eff_prob_equal100"].append(f"{len(eff_100)} ({(len(eff_100)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_prob_95"].append(f"{len(eff_95)} ({(len(eff_95)/len(origin_data_pred_df))*100:.1f}%)") # >95
        summary_dict["eff_90_prob_95"].append(f"{len(eff_90_95)} ({(len(eff_90_95)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_80_prob_90"].append(f"{len(eff_80_90)} ({(len(eff_80_90)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_70_prob_80"].append(f"{len(eff_70_80)} ({(len(eff_70_80)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_60_prob_70"].append(f"{len(eff_60_70)} ({(len(eff_60_70)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_50_prob_60"].append(f"{len(eff_50_60)} ({(len(eff_50_60)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_prob_equal50"].append(f"{len(eff_50)} ({(len(eff_50)/len(origin_data_pred_df))*100:.1f}%)")
        summary_dict["eff_50_prob"].append(f"{len(eff_low_50)} ({(len(eff_low_50)/len(origin_data_pred_df))*100:.1f}%)")
    
summary_df = pd.DataFrame(summary_dict)

output_summary = f"{base_dir}random_forest/summary_rf_predictions_onPhytoProteomes.tsv"
with open(output_summary, "w") as output:
    output.write(f"""#Random_Forest\n#Stratified5foldCV\n#Shuffle=Truex1\n#GridSearch_of_best_n_estimators=best_n_trees\n""")
    summary_df.to_csv(output, sep="\t", index=False)

# APPLICATION TO CaPziziphi
Super recent publication on identification of a possible type of non-canoncally secreted proteins involved in the host-metabolism perturbation
Gao, X., Ren, Z., Zhao, W. et al. Candidatus Phytoplasma ziziphi encodes non-classically secreted proteins that suppress hypersensitive cell death response in Nicotiana benthamiana. Phytopathol Res 5, 11 (2023). https://doi.org/10.1186/s42483-023-00166-0

In [9]:
## unique feature table 

for mod in models_name:
    loaded_model = joblib.load(f"{base_dir}random_forest/on_dataset20230224/{mod}_features/rf_best_model.pkl")

    table_base_dir = f"{base_dir}random_forest/"
    origin_data_pred_df = pd.read_csv(f"{table_base_dir}on_Ziziphi20230404/pred/feature_table_{mod}.tsv",
                                      sep="\t",
                                      header=0)

    original_protein_names = origin_data_pred_df["name"] 
    original_protein_seq_id = origin_data_pred_df["seq_id"]

    data_pred_df = origin_data_pred_df[origin_data_pred_df.columns[1:]] # df without seq_id == ready for predictions

    # apply the model
    clf = loaded_model.predict(data_pred_df[list(data_pred_df.columns)[1:]])
    list_clf = list(clf)
    df_clf = pd.DataFrame(clf, columns=["clf"])

    # save the predictions
    df_clf.to_csv(f"{base_dir}/random_forest/on_Ziziphi20230404/rf_best_model_{mod}_predictions.tsv", 
                  sep="\t", 
                  index=False)

    # probability of class assignment 
    prob = loaded_model.predict_proba(data_pred_df[list(data_pred_df.columns)[1:]])
    df_prob = pd.DataFrame(prob, columns=["effector_prob", "non_effector_prob"])
    df_prob["seq_id"] = list(original_protein_seq_id)
    df_prob.to_csv(f"{base_dir}/random_forest/on_Ziziphi20230404/probability_of_predictions_{mod}.tsv",
                   sep="\t",
                   index=False)
    


    ## save effectors with probaility = confidence of prediction > 90 into a fasta file
    highly_eff = df_prob[df_prob["effector_prob"] > 0.9]
    with open(f"{base_dir}/random_forest/on_Ziziphi20230404/effector_predictions_{mod}_filt90.fasta", 
              "w") as filtered_seq:
        for record in SeqIO.parse(f"{base_dir}datasets/ziziphi/ziziphi_genebank_proteome_WPids.fasta", "fasta"):
            if record.id in list(highly_eff["seq_id"]):
                filtered_seq.write(f">{record.description}\n{str(record.seq)}\n")
                
    

    # save the summary of performances 
    with open(f"{base_dir}/random_forest/on_Ziziphi20230404/prediction_{mod}_summary.txt", 
              "w") as out:
        count_eff = list_clf.count('effector')
        count_noneff = list_clf.count('non_effector')
        out.write(f"AT\nModel: {mod}\nPredicted:\n{count_eff} effectors\n{count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/count_noneff)*100}")


    # save the predictions as they are
    df_predictions = pd.DataFrame({"clf": list_clf, "protein_id": origin_data_pred_df["seq_id"]})
    df_predictions.to_csv(f"{base_dir}/random_forest/on_Ziziphi20230404/predictions_{mod}.tsv", 
                          sep="\t",
                          index=False)

    print(f"""AT\nModel: {mod}\nPredicted:\n{count_eff} effectors
    {count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/len(origin_data_pred_df))*100}%""")



AT
Model: clumps
Predicted:
169 effectors
    508 non-effectors
Proportion of effectors/total proteins: 24.963072378138847%
AT
Model: std
Predicted:
167 effectors
    510 non-effectors
Proportion of effectors/total proteins: 24.66765140324963%
AT
Model: clumps_std
Predicted:
169 effectors
    508 non-effectors
Proportion of effectors/total proteins: 24.963072378138847%
